In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/indore-city-house-price-predection/IndoreHP_Sample.csv
/kaggle/input/indore-city-house-price-predection/IndoreHP_Test.csv
/kaggle/input/indore-city-house-price-predection/IndoreHP_Train.csv


In [2]:
import pandas as pd
import numpy as np

train = pd.read_csv('../input/indore-city-house-price-predection/IndoreHP_Train.csv')
test = pd.read_csv('../input/indore-city-house-price-predection/IndoreHP_Test.csv')

print(train.info())
print(train.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           350 non-null    int64  
 1   CRIM                         350 non-null    float64
 2   ZN                           350 non-null    float64
 3   INDUS                        350 non-null    float64
 4   RIVER_FLG                    350 non-null    int64  
 5   nitric oxides concentration  350 non-null    float64
 6   #rooms/dwelling              350 non-null    float64
 7   AGE                          350 non-null    float64
 8   DIS                          350 non-null    float64
 9   RAD                          350 non-null    int64  
 10  TAX                          350 non-null    int64  
 11  PTRATIO                      350 non-null    float64
 12  B                            350 non-null    float64
 13  LSTAT               

In [3]:
X=train.drop(columns=['MEDV'])
y=train['MEDV']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
test=scaler.transform(test)


<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [4]:
# Create and train the XGBoost model with hyperparameter tuning and cross-validation
from xgboost import XGBRegressor
model = XGBRegressor(
    random_state=42,
    tree_method='gpu_hist', #to use GPU acceleration
    objective='reg:squarederror',  # Use squared error for regression
    eval_metric='rmse',  # Root Mean Squared Error as the evaluation metric
)

In [7]:
param_space = {
    'max_depth': (3, 10),
    'subsample': (0.5, 1),
    'colsample_bytree': (0.5, 1),
    'learning_rate': (0.01, 0.3),
    'gamma': (0, 5),
    'alpha': (0, 5),
    'n_estimators': (100, 200, 500),
    'early_stopping_rounds':[10] # Stop training if performance doesn't improve for 10 rounds
}

In [8]:
from skopt import BayesSearchCV
# Perform hyperparameter tuning using BayesSearchCV with GPU support
bayes_search = BayesSearchCV(estimator=model, 
                             search_spaces=param_space, 
                             cv=5, 
                             scoring='neg_mean_squared_error', 
                             n_jobs=-1
                            )

In [9]:
from tqdm import tqdm
# Fit the model with tqdm for progress tracking
with tqdm(total=bayes_search.total_iterations) as pbar:
    def update_pbar(_):
        pbar.update(1)

    bayes_search.fit(
    X_train,
    y_train,
    eval_set=[(X_test,y_test)],
    callback=update_pbar)

  0%|          | 0/400 [00:00<?, ?it/s]<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


[0]	validation_0-rmse:24.07767
[0]	validation_0-rmse:24.04130
[1]	validation_0-rmse:22.07736
[1]	validation_0-rmse:22.02757
[2]	validation_0-rmse:20.21371
[2]	validation_0-rmse:20.21131
[3]	validation_0-rmse:18.60662
[3]	validation_0-rmse:18.63384
[4]	validation_0-rmse:17.18555
[4]	validation_0-rmse:17.20161
[5]	validation_0-rmse:15.86346
[5]	validation_0-rmse:15.93491
[6]	validation_0-rmse:14.71835
[6]	validation_0-rmse:14.73800
[7]	validation_0-rmse:13.62865
[7]	validation_0-rmse:13.66830
[8]	validation_0-rmse:12.61736
[8]	validation_0-rmse:12.64619
[9]	validation_0-rmse:11.64139
[9]	validation_0-rmse:11.67654
[10]	validation_0-rmse:10.77424
[10]	validation_0-rmse:10.85045
[11]	validation_0-rmse:10.08425
[11]	validation_0-rmse:10.02212
[12]	validation_0-rmse:9.47708
[12]	validation_0-rmse:9.38172
[13]	validation_0-rmse:8.82803
[13]	validation_0-rmse:8.74095
[14]	validation_0-rmse:8.27059
[15]	validation_0-rmse:7.77220
[14]	validation_0-rmse:8.17050
[16]	validation_0-rmse:7.31312
[15]

  0%|          | 1/400 [00:04<30:36,  4.60s/it]

[84]	validation_0-rmse:3.02117
[85]	validation_0-rmse:3.02238
[86]	validation_0-rmse:3.01905
[87]	validation_0-rmse:3.02063
[88]	validation_0-rmse:3.01913
[89]	validation_0-rmse:3.01901
[90]	validation_0-rmse:3.01730
[91]	validation_0-rmse:3.02195
[92]	validation_0-rmse:3.02280
[93]	validation_0-rmse:3.02267
[94]	validation_0-rmse:3.02454
[95]	validation_0-rmse:3.02435
[96]	validation_0-rmse:3.02435
[97]	validation_0-rmse:3.02435
[98]	validation_0-rmse:3.02646
[99]	validation_0-rmse:3.02417
[0]	validation_0-rmse:20.79214
[1]	validation_0-rmse:16.54245
[2]	validation_0-rmse:13.53757
[3]	validation_0-rmse:11.21250
[0]	validation_0-rmse:20.94219
[4]	validation_0-rmse:9.30875
[1]	validation_0-rmse:16.54651
[5]	validation_0-rmse:8.04763
[2]	validation_0-rmse:13.20073
[6]	validation_0-rmse:7.14640
[3]	validation_0-rmse:10.68865
[7]	validation_0-rmse:6.37099
[4]	validation_0-rmse:8.83353
[8]	validation_0-rmse:5.72986
[5]	validation_0-rmse:7.37264
[9]	validation_0-rmse:5.30084
[10]	validation_

  0%|          | 2/400 [00:05<15:48,  2.38s/it]

...

 12%|█▏        | 49/400 [04:40<44:53,  7.67s/it]

[0]	validation_0-rmse:24.91879
[0]	validation_0-rmse:24.93652
[1]	validation_0-rmse:23.64913
[1]	validation_0-rmse:23.67879
[2]	validation_0-rmse:22.44999
[2]	validation_0-rmse:22.47485
[3]	validation_0-rmse:21.38111
[3]	validation_0-rmse:21.38701
[4]	validation_0-rmse:20.33639
[4]	validation_0-rmse:20.38906
[5]	validation_0-rmse:19.38023
[5]	validation_0-rmse:19.41073
[6]	validation_0-rmse:18.46289
[6]	validation_0-rmse:18.49363
[7]	validation_0-rmse:17.63508
[7]	validation_0-rmse:17.62146
[8]	validation_0-rmse:16.81164
[8]	validation_0-rmse:16.78218
[9]	validation_0-rmse:16.03184
[9]	validation_0-rmse:16.01290
[10]	validation_0-rmse:15.26773
[10]	validation_0-rmse:15.24283
[11]	validation_0-rmse:14.58881
[11]	validation_0-rmse:14.56210
[12]	validation_0-rmse:13.97708
[12]	validation_0-rmse:13.96531
[13]	validation_0-rmse:13.37267
[14]	validation_0-rmse:12.79460
[13]	validation_0-rmse:13.30166
[15]	validation_0-rmse:12.23219
[14]	validation_0-rmse:12.71069
[16]	validation_0-rmse:11.71

 12%|█▎        | 50/400 [04:49<45:47,  7.85s/it]

[0]	validation_0-rmse:25.51655
[1]	validation_0-rmse:24.79441
[2]	validation_0-rmse:24.10272
[3]	validation_0-rmse:23.39487
[4]	validation_0-rmse:22.74938
[5]	validation_0-rmse:22.09251
[6]	validation_0-rmse:21.48002
[7]	validation_0-rmse:20.89713
[8]	validation_0-rmse:20.30370
[9]	validation_0-rmse:19.74761
[10]	validation_0-rmse:19.20465
[11]	validation_0-rmse:18.66942
[12]	validation_0-rmse:18.16864
[13]	validation_0-rmse:17.61127
[14]	validation_0-rmse:17.14186
[15]	validation_0-rmse:16.67902
[16]	validation_0-rmse:16.21073
[17]	validation_0-rmse:15.78878
[18]	validation_0-rmse:15.36157
[19]	validation_0-rmse:14.95642
[20]	validation_0-rmse:14.56312
[21]	validation_0-rmse:14.17300
[22]	validation_0-rmse:13.79759
[23]	validation_0-rmse:13.40677
[24]	validation_0-rmse:13.04302
[25]	validation_0-rmse:12.71585
[26]	validation_0-rmse:12.39398
[27]	validation_0-rmse:12.08368
[28]	validation_0-rmse:11.78230
[29]	validation_0-rmse:11.48647
[30]	validation_0-rmse:11.23556
[31]	validation_0-

 12%|█▎        | 50/400 [04:50<33:55,  5.82s/it]


In [11]:
# Get the best model and its hyperparameters
# Get the best model and its parameters
best_model = bayes_search.best_estimator_
best_params = bayes_search.best_params_

print("Best Hyperparameters:", best_params)

# Make predictions on the test set using the best model
y_pred = best_model.predict(X_test)

from sklearn.metrics import mean_squared_error, mean_absolute_error
# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

Best Hyperparameters: OrderedDict([('alpha', 5), ('colsample_bytree', 0.5), ('early_stopping_rounds', 10), ('gamma', 5), ('learning_rate', 0.03148640519683524), ('max_depth', 7), ('n_estimators', 500), ('subsample', 0.5)])
Mean Squared Error: 8.7435027205009
Mean Absolute Error: 2.116586000932966


In [12]:
y_pred1=best_model.predict(test)

submission=pd.read_csv('../input/indore-city-house-price-predection/IndoreHP_Sample.csv')
submission['MEDV']=y_pred1
submission.to_csv('./submission2.csv', index=False)
submission.head()

,ID,MEDV
0,400,16.996277
1,401,21.002840
2,402,27.387955
3,403,17.956396
4,404,26.331654
